In [666]:
import pandas as pd
from collections import defaultdict
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
from keras.layers import Flatten

In [23]:
# does not contain 'H' or 'AB' data
condensed_df = pd.read_csv('src/data/condensed.csv')

In [26]:
condensed_df.columns

Index(['playerID', 'year1_2B', 'year1_3B', 'year1_BB', 'year1_CS', 'year1_G',
       'year1_GIDP', 'year1_HBP', 'year1_HR', 'year1_IBB',
       ...
       'year7_SF', 'year7_SH', 'year7_SO', 'year7_avg', 'pos_1B', 'pos_2B',
       'pos_3B', 'pos_C', 'pos_OF', 'pos_SS'],
      dtype='object', length=119)

In [27]:
six_year_cols = [col for col in condensed_df.columns if 'year7' not in col]
six_year_df = condensed_df[six_year_cols]
y = condensed_df['year7_avg']

In [82]:
six_year_df2 = six_year_df.astype(float)

In [439]:
uncondensed_df = pd.read_csv('src/data/uncondensed.csv')
uncondensed_df.drop('Unnamed: 0', axis=1, inplace=True)
uncondensed_df = pd.get_dummies(uncondensed_df, columns=['pos'])

In [440]:
uncondensed_df.drop('yearID', axis=1, inplace=True)
uncondensed_df.set_index('playerID', inplace=True)

In [441]:
uncondensed_df = uncondensed_df.astype(float)
uncondensed_df

,G,R,2B,3B,HR,RBI,SB,CS,BB,SO,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,122.0,58.0,27.0,6.0,13.0,69.0,2.0,2.0,28.0,39.0,...,6.0,4.0,13.0,0.279915,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,153.0,105.0,37.0,9.0,27.0,106.0,3.0,1.0,49.0,61.0,...,7.0,4.0,20.0,0.313953,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,153.0,106.0,34.0,14.0,26.0,92.0,2.0,4.0,37.0,54.0,...,5.0,7.0,21.0,0.328407,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,151.0,118.0,27.0,6.0,44.0,132.0,1.0,1.0,57.0,58.0,...,0.0,3.0,13.0,0.321951,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,153.0,109.0,34.0,4.0,30.0,95.0,4.0,1.0,59.0,49.0,...,0.0,3.0,21.0,0.326123,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,154.0,116.0,46.0,7.0,39.0,123.0,8.0,0.0,51.0,54.0,...,0.0,9.0,19.0,0.354531,0.0,0.0,0.0,0.0,1.0,0.0
aaronha01,153.0,102.0,20.0,11.0,40.0,126.0,16.0,7.0,60.0,63.0,...,0.0,12.0,8.0,0.291525,0.0,0.0,0.0,0.0,1.0,0.0
abbotku01,101.0,41.0,17.0,3.0,9.0,33.0,3.0,0.0,16.0,98.0,...,3.0,2.0,5.0,0.249275,0.0,0.0,0.0,0.0,0.0,1.0
abbotku01,120.0,60.0,18.0,7.0,17.0,60.0,4.0,3.0,36.0,110.0,...,2.0,5.0,6.0,0.254762,0.0,0.0,0.0,0.0,0.0,1.0


In [404]:
# X = uncondensed_df.groupby('playerID').apply(lambda x: x.iloc[:-1, :]).reset_index(0, drop=True)

# y = uncondensed_df.groupby('playerID').apply(lambda x: x.iloc[-1, :])

# y = pd.DataFrame(y['avg'])
# y

,avg
playerID,
aaronha01,0.291525
abbotku01,0.216561
abreubo01,0.299827
adairje01,0.270531
adamsbo03,0.282575
adamssp01,0.260204
adcocjo01,0.290749
ageeto01,0.227488
ainsmed01,0.191429


In [443]:
y_values = uncondensed_df.pop('avg')

In [444]:
X_panel = pd.Panel(dict_convert(uncondensed_df))
y_panel = pd.Panel(dict_convert(pd.DataFrame(y_values)))

In [471]:
X_panel['year1'].T

,aaronha01,abbotku01,abreubo01,adairje01,adamsbo03,adamssp01,adcocjo01,ageeto01,ainsmed01,aldremi01,...,zaungr01,zeidero01,zeileto01,zernigu01,zimmech01,zimmedo01,zimmehe01,zimmery01,ziskri01,zobribe01
2B,27.0,17.0,10.0,21.0,13.0,12.0,16.0,27.0,1.0,18.0,...,5.0,9.0,25.0,17.0,11.0,10.0,4.0,47.0,23.0,6.0
3B,6.0,3.0,2.0,1.0,3.0,0.0,1.0,8.0,2.0,3.0,...,0.0,2.0,3.0,2.0,4.0,1.0,1.0,3.0,7.0,2.0
BB,28.0,16.0,21.0,35.0,18.0,26.0,24.0,41.0,6.0,33.0,...,16.0,62.0,67.0,15.0,18.0,19.0,1.0,61.0,21.0,10.0
CS,2.0,0.0,2.0,2.0,0.0,19.0,0.0,18.0,0.0,3.0,...,1.0,0.0,4.0,1.0,0.0,3.0,0.0,8.0,0.0,3.0
G,122.0,101.0,59.0,133.0,94.0,95.0,102.0,160.0,33.0,84.0,...,40.0,136.0,144.0,73.0,65.0,88.0,46.0,157.0,103.0,52.0
GIDP,13.0,5.0,0.0,6.0,7.0,0.0,12.0,17.0,0.0,3.0,...,2.0,0.0,11.0,2.0,0.0,7.0,0.0,15.0,8.0,2.0
HBP,3.0,5.0,1.0,2.0,3.0,1.0,0.0,10.0,0.0,2.0,...,0.0,1.0,2.0,0.0,4.0,2.0,0.0,2.0,0.0,0.0
HR,13.0,9.0,3.0,9.0,4.0,4.0,8.0,22.0,0.0,2.0,...,3.0,0.0,15.0,5.0,0.0,15.0,0.0,20.0,10.0,2.0
IBB,0.0,1.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,4.0,...,0.0,0.0,3.0,0.0,0.0,5.0,0.0,7.0,0.0,1.0
R,58.0,41.0,22.0,41.0,35.0,40.0,46.0,98.0,4.0,27.0,...,18.0,57.0,62.0,29.0,27.0,38.0,17.0,84.0,44.0,10.0


In [446]:
X_data = X_panel.as_matrix().astype('float32')
y_data = y_panel.as_matrix()
print(X_data.shape, y_data.shape)

(7, 2348, 21) (7, 2348, 1)


In [447]:
# scaler = MinMaxScaler(feature_range=(0, 2))
# data_norm = scaler.fit_transform(dataset.reshape(dataset.shape[0], -1)).reshape(dataset.shape)

In [448]:
X_new_data = np.moveaxis(X_data, 0, 1)
y_new_data = np.moveaxis(y_data, 0, 1)
print(X_new_data.shape, y_new_data.shape)

(2348, 7, 21) (2348, 7, 1)


In [410]:
X_new_data[0, :, 0]

array([27., 37., 34., 27., 34., 46., nan], dtype=float32)

In [449]:
X_train, X_test, y_train, y_test = train_test_split(X_new_data, y_new_data)

In [450]:
y_test

array([[[0.28818444],
        [0.28705882],
        [0.2690678 ],
        ...,
        [0.26908397],
        [0.28762542],
        [0.2244489 ]],

       [[0.28150573],
        [0.24525316],
        [0.25988701],
        ...,
        [0.28692699],
        [0.23333333],
        [0.21988528]],

       [[0.2244898 ],
        [0.2962963 ],
        [0.22358722],
        ...,
        [0.2251816 ],
        [0.20740741],
        [0.2816092 ]],

       ...,

       [[0.29064039],
        [0.26259947],
        [0.29650092],
        ...,
        [0.30167598],
        [0.28729282],
        [0.31165919]],

       [[0.25619835],
        [0.29133858],
        [0.26      ],
        ...,
        [0.23370787],
        [0.26262626],
        [0.2195122 ]],

       [[0.3125    ],
        [0.27740492],
        [0.26130653],
        ...,
        [0.27130435],
        [0.24587156],
        [0.30927835]]])

In [451]:
y_train = y_train[:,6,:] # grab only the last column of BAs
y_test = y_test[:,6,:]
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1761, 7, 21)
(1761, 1)
(587, 7, 21)
(587, 1)


In [453]:
model = Sequential()
model.add(LSTM(4, input_shape=(7, 21)))
model.add(Dense(7))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, verbose=1, epochs=5, batch_size=1)

Epoch 1/5
1761/1761 [==============================] - 12s 7ms/step - loss: 0.0037
Epoch 2/5
1761/1761 [==============================] - 11s 6ms/step - loss: 0.0014
Epoch 3/5
1761/1761 [==============================] - 11s 6ms/step - loss: 0.0014
Epoch 4/5
1761/1761 [==============================] - 11s 6ms/step - loss: 0.0012
Epoch 5/5
1761/1761 [==============================] - 11s 6ms/step - loss: 0.0012


In [455]:
trainPred = model.predict(X_train)
testPred = model.predict(X_test)

print(trainPred.shape)
print(testPred.shape)

(1761, 1)
(587, 1)


In [456]:
print(y_test.shape)
print(testPred.shape)

(587, 1)
(587, 1)


In [457]:
testScore = math.sqrt(mean_squared_error(y_test, testPred))
trainScore = math.sqrt(mean_squared_error(y_train, trainPred))
print('Test Score: {:.3f}'.format(testScore))
print('Train Score: {:.3f}'.format(trainScore))

Test Score: 0.033
Train Score: 0.034


In [459]:
model.evaluate(X_test, y_test)

587/587 [==============================] - 1s 1ms/step


0.0011166594076697006

In [466]:
X_train[0,1,:]

array([ 15.,   2.,  25.,   4., 127.,   0.,   1.,   0.,   0.,  41.,  46.,
        13.,   0.,  11.,  27.,   0.,   0.,   0.,   0.,   0.,   1.],
      dtype=float32)

In [461]:
y_train

array([[0.24152542],
       [0.24017467],
       [0.30353818],
       ...,
       [0.27613941],
       [0.25384615],
       [0.24161074]])

# reincorporate BA
---

In [573]:
def dict_convert(df, length):
    def_dict = defaultdict(dict)
    if length == 1:
        return df.T.to_dict()
    for i in range(length):
        year = 'year' + str(i + 1)
        def_dict[year] = df.groupby('playerID').nth(i).T.to_dict('index')
    return def_dict

In [584]:
df = pd.read_csv('src/data/uncondensed.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df = pd.get_dummies(df, columns=['pos'])
df.set_index('playerID', inplace=True, drop=True)
df

,yearID,G,R,2B,3B,HR,RBI,SB,CS,BB,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,1954,122,58,27,6,13,69.0,2.0,2.0,28,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,1955,153,105,37,9,27,106.0,3.0,1.0,49,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,1956,153,106,34,14,26,92.0,2.0,4.0,37,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,1957,151,118,27,6,44,132.0,1.0,1.0,57,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,1958,153,109,34,4,30,95.0,4.0,1.0,59,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,1959,154,116,46,7,39,123.0,8.0,0.0,51,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0
aaronha01,1960,153,102,20,11,40,126.0,16.0,7.0,60,...,0.0,12.0,8.0,0.291525,0,0,0,0,1,0
abbotku01,1994,101,41,17,3,9,33.0,3.0,0.0,16,...,3.0,2.0,5.0,0.249275,0,0,0,0,0,1
abbotku01,1995,120,60,18,7,17,60.0,4.0,3.0,36,...,2.0,5.0,6.0,0.254762,0,0,0,0,0,1


In [585]:
X = df.groupby('playerID').head(6)

In [586]:
y = df['avg'].groupby('playerID').tail(6)
y = pd.DataFrame(y)

In [587]:
X

,yearID,G,R,2B,3B,HR,RBI,SB,CS,BB,...,SH,SF,GIDP,avg,pos_1B,pos_2B,pos_3B,pos_C,pos_OF,pos_SS
playerID,,,,,,,,,,,,,,,,,,,,,
aaronha01,1954,122,58,27,6,13,69.0,2.0,2.0,28,...,6.0,4.0,13.0,0.279915,0,0,0,0,1,0
aaronha01,1955,153,105,37,9,27,106.0,3.0,1.0,49,...,7.0,4.0,20.0,0.313953,0,0,0,0,1,0
aaronha01,1956,153,106,34,14,26,92.0,2.0,4.0,37,...,5.0,7.0,21.0,0.328407,0,0,0,0,1,0
aaronha01,1957,151,118,27,6,44,132.0,1.0,1.0,57,...,0.0,3.0,13.0,0.321951,0,0,0,0,1,0
aaronha01,1958,153,109,34,4,30,95.0,4.0,1.0,59,...,0.0,3.0,21.0,0.326123,0,0,0,0,1,0
aaronha01,1959,154,116,46,7,39,123.0,8.0,0.0,51,...,0.0,9.0,19.0,0.354531,0,0,0,0,1,0
abbotku01,1994,101,41,17,3,9,33.0,3.0,0.0,16,...,3.0,2.0,5.0,0.249275,0,0,0,0,0,1
abbotku01,1995,120,60,18,7,17,60.0,4.0,3.0,36,...,2.0,5.0,6.0,0.254762,0,0,0,0,0,1
abbotku01,1996,109,37,18,7,8,33.0,3.0,3.0,22,...,4.0,0.0,7.0,0.253125,0,0,0,0,0,1


In [588]:
X_dict = dict_convert(X, 6)
y_dict = dict_convert(y, 6)

In [589]:
X_panel1 = pd.Panel(X_dict)
y_panel1 = pd.Panel(y_dict)

print(X_panel1.shape)
print(y_panel1.shape)

(6, 2348, 23)
(6, 2348, 1)


In [592]:
X_matrix = X_panel1.as_matrix().astype('float32')
y_matrix = y_panel1.as_matrix()
print(X_matrix.shape)
print(y_matrix.shape)

(6, 2348, 23)
(6, 2348, 1)


In [593]:
X_matrix = np.moveaxis(X_matrix, 0, 1)
y_matrix = np.moveaxis(y_matrix, 0, 1)
print(X_matrix.shape)
print(y_matrix.shape)

(2348, 6, 23)
(2348, 6, 1)


In [596]:
X_train, X_test, y_train, y_test = train_test_split(X_matrix, y_matrix)
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train:\t{}\n'\
      'y_test:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

X_train:	(1761, 6, 23)
X_test:		(587, 6, 23)
y_train:	(1761, 6, 1)
y_test:		(587, 6, 1)



In [598]:
y_train1 = y_train[:,-1,:]
y_test1 = y_test[:,-1,:]
print('X_train:\t{}\n'\
      'X_test:\t\t{}\n'\
      'y_train1:\t{}\n'\
      'y_test1:\t\t{}\n'.format(X_train.shape, X_test.shape, y_train1.shape, y_test1.shape))

X_train:	(1761, 6, 23)
X_test:		(587, 6, 23)
y_train:	(1761, 1)
y_test:		(587, 1)



In [668]:
model1 = Sequential()
model1.add(LSTM(28, input_shape=(6, 23), return_sequences=True))
model1.add(BatchNormalization())
model1.add(Dropout(.3))
model1.add(LSTM(28))
model1.add(BatchNormalization())
model1.add(Dense(64))
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.fit(X_train, y_train1, verbose=1, epochs=3, batch_size=10)

Epoch 1/3
1761/1761 [==============================] - 5s 3ms/step - loss: 0.0241
Epoch 2/3
1761/1761 [==============================] - 2s 1ms/step - loss: 0.0042
Epoch 3/3
1761/1761 [==============================] - 2s 1ms/step - loss: 0.0038


In [669]:
testPred = model1.predict(X_test)
print(testPred.shape)

(587, 1)


In [670]:
model1.evaluate(X_test, y_test1)

587/587 [==============================] - 1s 2ms/step


0.0014954823901435737

In [671]:
X_test.shape

(587, 6, 23)

In [672]:
results = pd.DataFrame(y_test1)

In [673]:
results.columns = ['actual']

In [674]:
results['predicted'] = testPred

In [676]:
results['difference'] = results['actual'] - results['predicted']

In [678]:
results = results.round(3)

In [679]:
results

,actual,predicted,difference
0,0.283,0.253,0.030
1,0.230,0.290,-0.060
2,0.274,0.257,0.017
3,0.351,0.257,0.094
4,0.215,0.248,-0.033
5,0.223,0.255,-0.032
6,0.276,0.259,0.017
7,0.271,0.275,-0.003
8,0.237,0.257,-0.019
9,0.224,0.273,-0.049
